# Chess retrograde analysis
_Description_

Known Issues:
* Does not work with pawns on the board

## Imports

In [2]:
import chess
from anytree import Node, RenderTree
from math import factorial
from anytree.exporter import DotExporter

## Initial board entry
This will be replaced by @NicoMillers board setup code

In [3]:
fen = "8/8/8/8/4k3/8/8/K6R w - - 0 1"
board = chess.Board(fen)

## Setting up the piece list
The piece list contains all pieces on the board.

In [4]:
#get the piece map from the library
tmp_piece_map = board.piece_map()
pieceList = []
#iterate over the map and append the pieces to the list
for location in tmp_piece_map:
    pieceList.append(tmp_piece_map[location])

## Setting up the boardList
The board list is a list object containing 64! / (64-COUNT(Pieces))! empty boards to be filled later.

In [5]:
tmp_board = chess.Board()
tmp_board.clear()
board_list = []
list_length = factorial(64) // factorial(64 - len(pieceList))

for i in range(list_length):
    board_list.append(tmp_board.copy())

## The placePieces helper function
This function recursively places pieces on the board.
It takes the following arguments:
* boardList: A list containing empty or partially filled board objects
* pieceList: A list containing all pieces which should end up on the board
* pieceIndexStart: Index of the next piece to be placed
* start: range start for where to begin placing pieces in the boardList
* stop: range end for where to end placing pieces in the boardList

//TODO: Comments

In [6]:
def placePieces(boardList, pieceList, pieceIndexStart, start, stop):
    squareIndex = -1 #first Run increments to 0
    offset = (stop - start) // (64 - (len(pieceList) - 1) + pieceIndexStart)
    for i in range(start, stop):
        if i % offset == 0:
            if pieceIndexStart < (len(pieceList)-1) and i - offset > 0:
                placePieces(boardList, pieceList, pieceIndexStart + 1, i - offset, i)
            squareIndex = squareIndex + 1
        square = chess.Square(squareIndex)
        if boardList[i].piece_at(square) is not None:
            continue
        boardList[i].set_piece_at(square, pieceList[pieceIndexStart])

## Create all possible combinations of pieces on the board

In [7]:
placePieces(board_list, pieceList, 0, 0, len(board_list))

## Check boards for validity and filter out invalid ones

In [8]:
S = []
for board in board_list:
    #Board with white having the next turn
    board.turn = chess.WHITE
    status = board.status()
    if status == chess.Status.VALID or status == chess.Status.OPPOSITE_CHECK:
        S.append(board)
    boardBlackMove = board.copy()
    boardBlackMove.turn = chess.BLACK
    status = boardBlackMove.status()
    #The same board with black having the next turn
    if status == chess.Status.VALID or status == chess.Status.OPPOSITE_CHECK:
        S.append(boardBlackMove)

print(str(len(S)) + " valid boards")


468195 valid boards


## Find won boards and add them to S_0
Aditionally add them to S_0_ASCII for easier comparison in the next step.

In [9]:
S_tmp = []
S_0 = []
S_0_ASCII = []

for board in S:
    outcome = board.outcome()
    if outcome != None:
        if outcome.winner is not None:
            S_0.append(board)
            S_0_ASCII.append((board.turn,board.__str__()))
    else:
        S_tmp.append(board)
S = S_tmp

## Find all boards one step away from winning

In [16]:
S_1 = []

for board in S_0:
    for move in board.pseudo_legal_moves:
        board.push(move)
        if ((board.turn, board.__str__())) in   board.__eq__():
            continue
            #TODO - do Something

508


In [11]:
for board in S:
    for move in board.legal_moves:
        board.push(move)
        if (board.turn, board.__str__()) in S_0_ASCII:
            board.pop()
            S_1.append(board)
        else:
            board.pop()
    if S_1 != [] and board != S_1[-1]:
        S_tmp.append(board)
S = S_tmp

print(len(S_1))
print(len(S))

KeyboardInterrupt: 